In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import sys
sys.path.append("../")

In [2]:
from itertools import chain
import numpy as np
import networkx as nx
from tqdm import tqdm_notebook as tqdm
from CwnGraph import CwnBase

In [3]:
from MWE2019.cwn_vectors import CwnVectors
from MWE2019.cwn_morph_graph import CwnMorphGraph

In [4]:
cwn = CwnBase()

In [5]:
cwn_vec = CwnVectors()

load from cache:  E:\LangOn\MWE2019\data\cache_cwn_vectors\cwn_vectors.pkl


# Build CwnMorphGraph

In [24]:
cwn_mg = CwnMorphGraph()

In [26]:
G = cwn_mg.G

In [30]:
from collections import Counter
reltype_counts = Counter(edata["reltype"] for _, edata in G.edges.items())

In [31]:
reltype_counts

Counter({'synonym': 8715,
         'varword': 639,
         'antonym': 1333,
         'nearsynonym': 1348,
         'hyponym': 1263,
         'hypernym': 962,
         'morph': 13480,
         'meronym': 115,
         'holonym': 73,
         'variant': 4})